### Read all the data that's required

In [ ]:
%run "../includes/configurations"

In [ ]:
from pyspark.sql.functions import col, current_timestamp

In [ ]:
flights_df = (spark.read.parquet(f"{processed_folder_path}flights").
              filter(col("cancelled") == 0).
              select(
                  col("origin"),
                  col("destination"),
                  col("day_of_month"),
                  col("month"),
                  col("year"),
                  col("sched_dep_time"),
                  col("sched_arr_time"),
                  col("act_elapsed_time"),
                  col("flight_num"),
                  col("uniq_carrier_code"),
                  col("distance").
                  col("flight_id")
              )
             )

In [ ]:
display(flights_df)

In [ ]:
airports_df = (spark.read.parquet(f"{processed_folder_path}airports").
               select(
                   col("airport_name"),
                   col("iso_country"),
                   col("municipality"),
                   col("iata_code")
               ).
               withColumnRenamed("iso_country", "airport_country").
               withColumnRenamed("iata_code", "airport_code")
              )

In [ ]:
airlines_df = (spark.read.parquet(f"{processed_folder_path}airlines").
               select(col("airline_country"),col("name"),col("iata")).
               withColumnRenamed("name","airline_name").
               withColumnRenamed("iata","airline_code")
              )

### Filter Airports and Airlines in US

In [ ]:
airports_filtered_df = airports_df.filter(col("airport_country") == "US")

In [ ]:
display(airports_filtered_df)

In [ ]:
airlines_filtered_df = airlines_df.filter(col("airline_country") == "United States")

In [ ]:
display(airlines_filtered_df)

### Join Airlines to Flights

In [ ]:
flight_airlines_df = flights_df.join(
    airlines_filtered_df,
    flights_df.uniq_carrier_code == airlines_filtered_df.airline_code,
    "left"
)

### Join Airports to Flights and Airlines

In [ ]:
flight_origin_df = flight_airlines_df.join(
    (airports_filtered_df.select(
        col("airport_code"),
        col("airport_name"),
        col("municipality")
    ).
    withColumnRenamed("airport_name", "origin_airport").
    withColumnRenamed("municipality", "origin_municipality")
    ),
    flight_airlines_df.origin == airports_filtered_df.airport_code,
    "left"
)
flight_origin_df = flight_origin_df.drop("airport_code")

In [ ]:
display(flight_origin_df)

In [ ]:
flight_final_df = flight_origin_df.join(
    (airports_filtered_df.select(
        col("airport_code"),
        col("airport_name"),
        col("municipality")
    ).
    withColumnRenamed("airport_name", "dest_airport").
    withColumnRenamed("municipality", "dest_municipality")
    ),
    flight_origin_df.destination == airports_filtered_df.airport_code,
    "left"
)
flight_final_df = flight_final_df.drop("airport_code")

In [ ]:
display(flight_final_df)

In [ ]:
flight_final_df = flight_final_df.withColumn("created_date", current_timestamp())

In [ ]:
display(flight_final_df)

In [ ]:
# flight_final_df.write.mode("overwrite").partitionBy("year").parquet(f"{presentation_folder_path}search_flight")

In [ ]:
from src import utils as ut

In [ ]:
ut.overwrite_partition(flight_final_df, "dev_air_travel_present", "search_flight", "flight_id")

In [ ]:
# (flight_final_df.
#  write.mode("overwrite").
#  partitionBy("year").
#  format("parquet").
#  saveAsTable("dev_air_travel_present.search_flight")
# )

In [ ]:
# dbutils.fs.ls("/mnt/airlineblobstorage/")